In [ ]:
import numpy, pandas
from sqlalchemy import create_engine

dbConn = create_engine('mysql+pymysql://root:<DBPASSWORD>@localhost/epiviz')

Database measurements

In [ ]:
from measurements.measurementClass import DbMeasurement
# measuremment name, column name, table name, database name
dbm = DbMeasurement("db", "cancer", "cancer", "gene_expression", "epiviz", dbConn=dbConn)
result, _ = dbm.get_data("chr11", 30064187, 31184187)
result.describe()
# result.head()

In [ ]:
## bin rows

start = 30064187
end = 31184187
chr = 'chr11'
length = 2000
columns = ['chr', 'cancer']

# print("start-end = %s" %(end-start))

freq = round((end-start)/length)
if end - start < length:
    freq = 1
    
# print("freq = %s" % (freq))
    
data = result

data = data.set_index(['start', 'end'])
data.index = pandas.IntervalIndex.from_tuples(data.index)

bins = pandas.interval_range(start=start, end=end, freq=freq)
# print(bins)
bins_df = pandas.DataFrame(index=bins)
bins_df["chr"] = chr
# if self.metadata:
#     for meta in metadata:
#         bins_df[meta] = data[meta]

# print(bins_df)

for col in columns:
    bins_df[col] = None

# print(bins_df)

# map data to bins
for index, row in data.iterrows():
#     print(row)
#     print(index)
    # subset mapped rows
    for col in columns:
#         print(bins_df.loc[index])
        bins_df.loc[index, col] = row[col]

bins_df["start"] = bins_df.index.left
bins_df["end"] = bins_df.index.right
    
bins_df.describe()
    
    
# for col in ['cancer']:
#     print(data[col])
#     bins_df[col] = bins_df.index.map(data[col])


In [ ]:
test = bins_df.loc[[x for x in data.index]]
len(test)

File measurements

In [ ]:
from measurements.measurementClass import FileMeasurement
# file type, file id, file name, file source
filem = FileMeasurement("bigwig", "39033", "39033", "http://obj.umiacs.umd.edu/bigwig-files/39033.bigwig")
# result, _ = filem.get_data('chr8', 115443360, 115443460)
result, _ = filem.get_data("chr11", 30064187, 31184187)
result.describe()

In [ ]:
from measurements.measurementClass import ComputedMeasurement
computem = ComputedMeasurement("computed", "test_compute", "compute avg", measurements=[dbm, filem], computeFunc=numpy.sum)

result, _ = computem.get_data("chr11", 30064187, 31184187)
result

In [ ]:
from measurements.measurementClass import ComputedMeasurement
computem = ComputedMeasurement("computed", "test_compute", "compute avg", measurements=[dbm, filem], computeFunc=numpy.sum)
# result, _ = computem.get_data("chr11", 30064187, 31184187)

chr = "chr11"
start = 30064187
end = 31184187
measurements=[dbm, filem]
computeFunc = numpy.sum
columns = ["cancer", "39033"]

result = []
for measurement in measurements:
    mea_result, _ = measurement.get_data(chr, start, end, bin=True)
#     print(mea_result)
    # result = [result, mea_result]
    result.append(mea_result)
    
result = pandas.concat(result, axis=1)
result = result.loc[:,~result.columns.duplicated()]
# print(result.describe())
# print(result["cancer"].describe())
# print(result["39033"].describe())
# print(result.dropna())

def computeWrapper(computeFunc, columns):
    def computeApply(row):
#         print(row)
        rowVals = []
        for k in row.keys():
            if k in columns:
                rowVals.append(row[k])
#         print(rowVals)
        if None in rowVals:
            return None
        return computeFunc(rowVals)
    return computeApply
    
result = result.dropna()
result["compute"] = result.apply(computeWrapper(computeFunc, columns), axis=1)
result

In [ ]:
def compute(measurements, chr, start, end, fun=None):
    mresults = []
    for mc in measurements:
        mresults.append(mc.get_data(chr, start, end))
    return mresults

Computed measurements

In [ ]:
# measurements = [("db", "cancer", "cancer", "gene_expression", "epiviz"), ("bigwig", "39033", "39033", "http://obj.umiacs.umd.edu/bigwig-files/39033.bigwig")]
# cm = mc.ComputedMeasurement(measurements,fun=numpy.mean)

# from measurements.measurementClass import compute
measurements = [filem, dbm]

compute(measurements, "chr11", 30064187, 31184187, fun=numpy.mean)
